In [1]:
import requests
from bs4 import BeautifulSoup as soup
import os, time
import json,re
import random

In [58]:
main_url = "https://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;filter=advanced;orderby=runs;size=150;spanmin1=01+Jan+1980;spanval1=span;team=8;template=results;type=batting"

In [59]:
response = requests.get(
    url = main_url,
)

main_soup = soup(response.content, 'html.parser')

In [60]:
cricketers_list_raw = main_soup.find_all('tr', "data1")

In [61]:
cricketers_list = []
for i in range(0,100):
    for x in cricketers_list_raw[i].find_all("a", "data-link"):
        cricketers_list.append({"name": x.getText(), "url": x.get("href")})
    

In [62]:
# cricketers_list

In [63]:
base_url_player = "https://stats.espncricinfo.com/"

In [83]:
# Filter dictionary keeping necessary fields
def filter_dict(di, keys):
    if not isinstance(di, (dict, list)):
        return di
    if isinstance(di, list):
        return [filter_dict(value, keys) for value in di]
    return {key: filter_dict(value, keys) for key, value in di.items() if key in keys or isinstance(value, (dict, list))}


In [93]:
player_data_keys = ["Full Name", "Born", "Age", "Batting Style", "Bowling Style", "Playing Role", "Education"]
def extract_player_data(soup):
    player_card = soup.find_all('div', "player_overview-grid")
    player_details_raw = player_card[0].find_all('div')
    player_details = {}
    for detail in player_details_raw:
        if(detail.p.getText() == "Fielding Position"):
            continue
        player_details[detail.p.getText()] = detail.h5.getText()
    if(not "Bowling Style" in player_details):
        player_details["Bowling Style"] = "N/A"
    if(not "Playing Role" in player_details):
        player_details["Playing Role"] = "N/A"
    # print(player_details["Full Name"])
    return filter_dict(player_details, player_data_keys)
    

In [94]:
def extract_odi_data(soup):
    standings_card_raw = soup.find_all("div", "overflow-hidden")
    tables_raw_check = standings_card_raw[2].find_all("h5")
    
    # Finding Batting and Bowling tables and extracting the table
    for i in range (2):
        if(tables_raw_check[i].getText()=="Batting & Fielding"):
            batting_table_raw = tables_raw_check[i].find_next("table", "table standings-widget-table text-center mb-0 border-bottom")
        else:
            bowling_table_raw = tables_raw_check[i].find_next("table", "table standings-widget-table text-center mb-0 border-bottom")

    player_values = {}
    # Batting Values extraction
    # Finding ODI row and extracting the row
    batting_table_rows_raw = batting_table_raw.find("tbody").find_all("tr")
    for row in batting_table_rows_raw:
        if (row.find("span").getText()=="ODI"):
            batting_table_odi_row = row
            break
        else:
            continue
    batting_values_raw = batting_table_odi_row.find_all("span")
    player_values["Matches"] = batting_values_raw[1].getText()
    player_values["Runs"] = batting_values_raw[4].getText()
    player_values["Average"] = batting_values_raw[6].getText()
    player_values["Half-centuries"] = batting_values_raw[10].getText()
    player_values["Centuries"] = batting_values_raw[9].getText()

    # Bowling values extraction
    # Finding ODI row and extracting the row
    bowling_table_rows_raw = bowling_table_raw.find("tbody").find_all("tr")
    for row in bowling_table_rows_raw:
        if (row.find("span").getText()=="ODI"):
            bowling_table_odi_row = row
            break
        else:
            continue
    bowling_values_raw = bowling_table_odi_row.find_all("span")
    player_values["wickets"] =bowling_values_raw[5].getText() if(bowling_values_raw[5].getText()!="-") else "N/A"
    player_values["economy"] =bowling_values_raw[9].getText() if(bowling_values_raw[9].getText()!="-") else "N/A"
    return player_values
    

In [95]:
# Main Function
cricketers = []
for i in range(0,100):
    # print(i)
    player_url = base_url_player + cricketers_list[i]["url"]
    response = requests.get(
    url = player_url,
    )
    player_soup = soup(response.content, 'html.parser')
    player_data = extract_player_data(player_soup) | extract_odi_data(player_soup)
    cricketers.append(player_data)

In [87]:
# print(cricketers)

In [96]:
import pandas as pd

my_dict = [ {'type' : 'type_1', 'prob' : 2, 'x_sum' : 3, 'y_sum' : 5},
 {'type' : 'type_2', 'prob' : 3, 'x_sum' : 8, 'y_sum' : 6}]

df = pd.DataFrame(cricketers)
from tabulate import tabulate
pdtabulate=lambda df:tabulate(df,headers='keys',tablefmt='psql')
# print(pdtabulate(df))


In [97]:
cric = []
for cricketer in cricketers:
    cric.append({k: cricketer[k] for k in cricketer.keys() & {'Batting Style', 'Bowling Style', 'Playing Role'}})

s = set( val for dic in cric for val in dic.values())

# for x in s:                                
    # print (x)
# print (len(s)) 

In [ ]:
{"Left arm fast medium":"",
"Slow left arm orthodox":"",
"Right arm medium fast": "දකුණත් මද වේග පන්දු යවන",
"Legbreak googly": "", 
"Legbreak": "පා දඟ පන්දු යවන",
"Left arm medium fast": "වමත් මද වේග පන්දු යවන",
Top order batter - ඉදිරි පෙළ පිතිකරු
Right hand bat - දකුණත් පිතිකරු
Right arm offbreak, Legbreak - දකුණත් දඟ පන්දු හා පා දඟ පන්දු යවන
Batter - පිතිකරු
Middle order batter - මැද පෙළ පිතිකරු
Left hand bat - වමත් පිතිකරු
Bowling allrounder - පන්දු යවන තුන් ඉරියව් ක්‍රීඩක
Bowler -  පන්දු යවන්නා
Wicketkeeper batter - කඩුලු රකින පිතිකරු
Batting allrounder - පිතිකරණ තුන් ඉරියව් ක්‍රීඩක
N/A
Allrounder - තුන් ඉරියව් ක්‍රීඩක
Right arm medium - දකුණත් මද පන්දු යවන
Left arm medium, Slow left arm orthodox - 
Right arm offbreak - දකුණත් දඟ පන්දු යවන
Right arm fast - දකුණත් වේග පන්දු යවන
Opening batter - ආරම්භක පිතිකරු
Right arm fast medium -}